In [7]:
from pyspark.sql import SparkSession, functions as F, Window as W
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField,ArrayType,LongType,StringType,DoubleType,IntegerType
from pyspark.sql.functions import split, explode,col
from pyspark.sql.functions import lit


spark = SparkSession.builder.appName('earthquake_api').getOrCreate()

In [2]:
earthquake_df = spark.read\
    .format('json')\
    .option('multiLine', 'true')\
    .load('data.json')

In [3]:
earthquake_df.printSchema()

root
 |-- count: long (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- alert: string (nullable = true)
 |    |    |-- cdi: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- continent: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |    |-- depth: string (nullable = true)
 |    |    |-- detailUrl: string (nullable = true)
 |    |    |-- dmin: string (nullable = true)
 |    |    |-- felt: string (nullable = true)
 |    |    |-- gap: string (nullable = true)
 |    |    |-- geometryType: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- ids: string (nullable = true)
 |    |    |-- latitude: string (nullable = true)
 |    |    |-- locality: string (nullable = true)
 |    |    |-- location: string (nullable = true)
 |    |    |-- loca

In [7]:
# number of rows
earthquake_df.count()

1

In [8]:
earthquake_df.show()

+-----+--------------------+---------+------+-------------+----------+-----------+-------+----+
|count|                data|errorCode|errors|friendlyError|httpStatus|       noun| result|verb|
+-----+--------------------+---------+------+-------------+----------+-----------+-------+----+
|  100|[{, 3, , 7000iqb8...|     none|    []|             |       200|earthquakes|success| GET|
+-----+--------------------+---------+------+-------------+----------+-----------+-------+----+



In [4]:
earthquake_df = earthquake_df.drop("count").drop("errorCode").drop("errors").drop("friendlyError").drop("httpStatus").drop("noun").drop( "result").drop("verb")


In [10]:
earthquake_df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- alert: string (nullable = true)
 |    |    |-- cdi: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- continent: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |    |-- depth: string (nullable = true)
 |    |    |-- detailUrl: string (nullable = true)
 |    |    |-- dmin: string (nullable = true)
 |    |    |-- felt: string (nullable = true)
 |    |    |-- gap: string (nullable = true)
 |    |    |-- geometryType: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- ids: string (nullable = true)
 |    |    |-- latitude: string (nullable = true)
 |    |    |-- locality: string (nullable = true)
 |    |    |-- location: string (nullable = true)
 |    |    |-- locationDetails: array (nullable = true

In [5]:
earthquake_df = earthquake_df.withColumn('data', F.explode('data'))

In [6]:
earthquake_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- alert: string (nullable = true)
 |    |-- cdi: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- continent: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- depth: string (nullable = true)
 |    |-- detailUrl: string (nullable = true)
 |    |-- dmin: string (nullable = true)
 |    |-- felt: string (nullable = true)
 |    |-- gap: string (nullable = true)
 |    |-- geometryType: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- ids: string (nullable = true)
 |    |-- latitude: string (nullable = true)
 |    |-- locality: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- locationDetails: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- adminLevel: string (nullable = true)
 |    |    |    |-- descriptio

In [18]:
earthquake_df = earthquake_df\
            .withColumn('id', F.col('data').getItem('id'))\
            .withColumn('magnitude', F.col('data').getItem('magnitude').cast('int'))\
            .withColumn('type', F.col('data').getItem('type'))\
            .withColumn('title', F.col('data').getItem('title'))\
    	    .withColumn('date', F.col('data').getItem('date'))\
    	    .withColumn('time', F.col('data').getItem('time'))\
    	    .withColumn('updated', F.col('data').getItem('updated').cast('int'))\
    	    .withColumn('status', F.col('data').getItem('status'))\
    	    .withColumn('latitude', F.col('data').getItem('latitude').cast('int'))\
    	    .withColumn('longitude', F.col('data').getItem('longitude').cast('int'))\
    	    .withColumn('place', F.col('data').getItem('place'))\
    	    .withColumn('location', F.col('data').getItem('location'))\
    	    .withColumn('continent', F.col('data').getItem('continent'))\
    	    .withColumn('country', F.col('data').getItem('country'))
           



AnalysisException: Column 'data' does not exist. Did you mean one of the following? [date, id, place, status, time, title, type, updated, country, latitude, location, magnitude, continent, longitude];
'Project ['data[id] AS id#572, magnitude#234, type#251, title#268, date#285, time#302, updated#319, status#336, latitude#353, longitude#370, place#387, location#404, continent#421, country#438]
+- Project [id#217, magnitude#234, type#251, title#268, date#285, time#302, updated#319, status#336, latitude#353, longitude#370, place#387, location#404, continent#421, country#438]
   +- Project [data#56, id#217, magnitude#234, type#251, title#268, date#285, time#302, updated#319, status#336, latitude#353, longitude#370, place#387, location#404, continent#421, country#438, 100 AS counts#479]
      +- Project [data#56, id#217, magnitude#234, type#251, title#268, date#285, time#302, updated#319, status#336, latitude#353, longitude#370, place#387, location#404, continent#421, data#56.country AS coun

In [11]:
earthquake_df = earthquake_df\
                .select('id','magnitude', 'type', 'title', 'date', 'time', 'updated', 'status', 'latitude','longitude', 'place','location','continent','country')


In [13]:
earthquake_df.select(earthquake_df['counts']).show()

AnalysisException: Cannot resolve column name "counts" among (id, magnitude, type, title, date, time, updated, status, latitude, longitude, place, location, continent, country)

In [9]:
earthquake_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- alert: string (nullable = true)
 |    |-- cdi: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- continent: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- depth: string (nullable = true)
 |    |-- detailUrl: string (nullable = true)
 |    |-- dmin: string (nullable = true)
 |    |-- felt: string (nullable = true)
 |    |-- gap: string (nullable = true)
 |    |-- geometryType: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- ids: string (nullable = true)
 |    |-- latitude: string (nullable = true)
 |    |-- locality: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- locationDetails: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- adminLevel: string (nullable = true)
 |    |    |    |-- descriptio

In [16]:
# number of rows
earthquake_df.count()

100

In [ ]:
# Transformation 
# 1. Find the average of magnitude of earthquake.

In [17]:
average_magnitude_df = earthquake_df.agg({'magnitude':'avg'})
average_magnitude_df.select(average_magnitude_df['avg(magnitude)'])
average_magnitude_df.show()

+--------------+
|avg(magnitude)|
+--------------+
|          3.36|
+--------------+



In [22]:
average_magnitude_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/kafka', driver= 'org.postgresql.Driver',
                                                    dbtable='earthquake_table_1', user='memosha',password='1234').mode('overwrite').save()

Py4JJavaError: An error occurred while calling o173.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:229)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:233)
	at or

In [ ]:
# 2. Highest magnitude for each country in the month of November in 2022.

In [19]:
highest_magnitude_november_df = earthquake_df\
                    .groupBy(F.year('date').alias('year'), F.month('date').alias('month'), 'location')\
                    .agg(F.max('magnitude').alias('highest_magnitude'))\
                    .filter('month == 11')\
                    .filter('year == 2022')\
                    .orderBy('location', 'year')

highest_magnitude_november_df.show()

+----+-----+--------------------+-----------------+
|year|month|            location|highest_magnitude|
+----+-----+--------------------+-----------------+
|2022|   11|Bahía de Kino, Me...|                6|
|2022|   11|Colorado City, Ar...|                3|
|2022|   11|       Corinne, Utah|                3|
|2022|   11|     Dubois, Wyoming|                3|
|2022|   11|  Lake Pillsbury, CA|                3|
|2022|   11| Maneadero, B.C., MX|                4|
|2022|   11|      Mentone, Texas|                5|
|2022|   11|    Ravalli, Montana|                3|
|2022|   11|      Stanley, Idaho|                3|
|2022|   11|     Taylor, Wyoming|                3|
|2022|   11|        Toyah, Texas|                3|
|2022|   11|Whites City, New ...|                3|
+----+-----+--------------------+-----------------+



In [20]:
highest_magnitude_november_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/kafka', driver= 'org.postgresql.Driver',
dbtable='earthquake_table_1', user='memosha',password='1234').mode('overwrite').save()

Py4JJavaError: An error occurred while calling o155.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:229)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:233)
	at or

In [ ]:
# 3. Find the status of earthquake on the basis of each place.

In [75]:
status_place_df = earthquake_df.withColumn('status',F.collect_set('status').over(W.partitionBy('place'))).select('place', 'status').distinct()
status_place_df.show()

+--------------------+----------+
|               place|    status|
+--------------------+----------+
|100km S of San Cl...|[reviewed]|
|12km WSW of Petro...|[reviewed]|
|13km NNW of Grape...|[reviewed]|
|13km SE of Bodfis...|[reviewed]|
|14km ESE of Alum ...|[reviewed]|
|14km ESE of Woffo...|[reviewed]|
|14km SE of Baysid...|[reviewed]|
|15km E of Seven T...|[reviewed]|
|15km ESE of Alum ...|[reviewed]|
|16 km ESE of Laco...|[reviewed]|
|16km WNW of Lake ...|[reviewed]|
|16km WSW of Weitc...|[reviewed]|
|17 km NNW of Tayl...|[reviewed]|
|21 km NW of Bickl...|[reviewed]|
|21 km SSE of Golf...|[reviewed]|
|22 km SSW of Mamm...|[reviewed]|
|23km E of Julian, CA|[reviewed]|
|23km NE of Therma...|[reviewed]|
|24 km NW of Stanl...|[reviewed]|
|25km W of Petroli...|[reviewed]|
+--------------------+----------+
only showing top 20 rows

